In [1]:

import pandas as pd

In [4]:

# this dataset pertains to veris
# this will help us identify how the attacker will target organization
# it categorizes ttps into various as per below list
# hacking, malware, integrity, confidentiality, availability, social, valuechain(persistant)

veris_df  = pd.read_csv('../data/veris_attack_mapping.csv')
veris_impact_df  = pd.read_csv('../score/veris_impact.csv')

veris_df = veris_df.drop(veris_df[veris_df['mapping_type']=='non_mappable'].index)

veris_df = veris_df.drop(columns=['attack_version',
                                'technology_domain',
                                'mapping_type',
                                'references',
                                'comments',
                                'organization',
                                'creation_date',
                                'last_update',
                                'mapping_framework_version',
                                'mapping_framework', 'Unnamed: 0'])
veris_impact_df = veris_impact_df.drop(columns=['description', 'id'])
# display(veris_impact_df.columns)
# display(veris_df)

# Create one DataFrame with "action"
veris_df_action = veris_df[veris_df['capability_group'].str.contains('action')]

veris_df_action.loc[:, 'capability_group'] = veris_df_action['capability_group'].str.replace('action.', '', regex=False)
veris_df_action.loc[:, 'capability_group'] = veris_df_action['capability_group'].str.replace('attribute.', '', regex=False)

veris_df_action.loc[:, 'capability_id'] = veris_df_action['capability_id'].str.replace(r'action\.\w+\.(variety|vector)\.', '', regex=True)

veris_df_action = pd.merge(veris_df_action, veris_impact_df, left_on='capability_id', right_on='attack_type', how='left')

veris_df_action.drop(columns=['capability_id'], inplace=True)


# Create another DataFrame with "attribute"

veris_df_attribute = veris_df[veris_df['capability_group'].str.contains('attribute')]
veris_df_attribute.loc[:, 'capability_group'] = veris_df_attribute['capability_group'].str.replace('attribute.', '', regex=False)



pd.set_option('display.max_rows', None)

# display(veris_df_action)
# display(veris_df_attribute.head())


ttps = ['T1136', 'T1133', 'T1543']

#veris data

veris_df = veris_df_action.loc[veris_df_action['attack_object_id'].isin(ttps)].reset_index(drop=True)


veris_df_attribute = veris_df_attribute.loc[veris_df_attribute['attack_object_id'].isin(ttps)].reset_index(drop=True)

# Group by attack_object_id and calculate the average severity
average_severity = veris_df.groupby('attack_object_id')['severity'].mean().reset_index()

# Define severity categories
bins = [0, 5, 7, 9, 10]  # Define the bins
labels = ['Low', 'Moderate', 'High', 'Critical']  # Define the labels


# Rename columns for clarity
# Rename columns for clarity
average_severity.columns = ['ttp', 'severity']

# Create a new column for severity levels
average_severity['severity_level'] = pd.cut(average_severity['severity'], bins=bins, labels=labels, right=True)

# Count occurrences of each severity level and reset index to convert it into a DataFrame
severity_counts = average_severity['severity_level'].value_counts().reset_index()

# Rename columns for clarity after resetting index
severity_counts.columns = ['severity_level', 'count']

# Define the custom order for severity levels
severity_order = pd.CategoricalDtype(['Low', 'Moderate', 'High', 'Critical'], ordered=True)

# Convert the severity_level column to this custom order
severity_counts['severity_level'] = severity_counts['severity_level'].astype(severity_order)

# Sort the DataFrame based on severity_level
severity_counts = severity_counts.sort_values('severity_level', ignore_index=True)

# Display the sorted DataFrame
print(severity_counts)
# Display the result
display(average_severity)
# display(severity_counts)

# whether they target confidentiality integrity or availability
# display(veris_df)
display(veris_df_attribute.groupby('capability_group')['capability_id'].count().reset_index())

  severity_level  count
0            Low      0
1       Moderate      0
2           High      3
3       Critical      0


,ttp,severity,severity_level
0,T1133,7.100000,High
1,T1136,8.333333,High
2,T1543,8.166667,High


,capability_group,capability_id
0,integrity,2
